In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd

from time import sleep
from time import time 

import random

from IPython.core.display import clear_output

In [ ]:
# Set the file name for save web data
file_name = 'jobdata2.xlsx'

# Set page numbers for scraping
start_page = 151
end_page = 200


# define the search condition, url link and head
search_word = '/data'
main_url = "******"
if start_page == 1:
    seek_url = main_url + search_word + "-jobs"
else:
    seek_url = main_url + search_word + "-jobs" + '?page='+ str(start_page)
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
    
# Making a get request to fetch the first webpage 
seek_page = requests.get(seek_url, headers = header)
# Parsing the first webpage using BeautifulSoup
seek_soup = bs(seek_page.text,"lxml")

# Preparing the monitoring of the loop
start_time = time()
count = 1

for page in range(start_page,end_page+1):
    print(seek_url)
    # A list to save all searched jobs information: 
    #job_url, job title, company, job location, salary and job description
    job_data = []
    
    flag = 0
    ## Locating the position of every searched results
    all_jobs = seek_soup.find_all("div",attrs={"data-search-sol-meta":True})

    for job in all_jobs:
        job_url = main_url + job.h1.a["href"]
        job_title = job.h1.a.text
        try:
            job_company = job.find("a",attrs={"data-automation":"jobCompany"}).text
        except:
            job_company = None

        job_location = job.find(text=re.compile("location"))
        try:
            job_salary = job.find("span",attrs={"data-automation":"jobSalary"})["aria-label"]
        except:
            job_salary = None
        job_data.append([job_url, job_title, job_company,job_location,job_salary])
        print(flag)
        flag +=1
        

    for result in job_data:
        # Fetching each job page, to get detailed job info
        job_page = requests.get(result[0],timeout = 10, headers = header)
        # extract
        job_soup = bs(job_page.text,"lxml") 

        # Job published date, industry and job infromation
        job_date = job_soup.find_all("span",class_="lwHBT6d")[1].text
        job_indus = job_soup.select("dl strong")[1].text
        job_info = job_soup.find("div",attrs={"data-automation":"mobileTemplate"}).text
        result.append(job_date)
        result.append(job_indus)
        result.append(job_info)
            # Pause the loop
        sleep(random.randint(2,10))
    
    # Convert data to Pandas format
    column = ['job_url','job_title', 'job_company','job_location','job_salary','job_date','job_indus', 'job_info']
    job_df = pd.DataFrame(job_data, columns = column)

    # Write into excel
    # Create a Pandas Excel writer 
    writer = pd.ExcelWriter(file_name,engine=None,mode = 'a')
    sheetname = 'Sheet'+ str(page)

     # Write the dataframe with all information of one webpage to one sheet of an XlsxWriter Excel object.
    job_df.to_excel(writer, sheet_name=sheetname)
     # Close the Pandas Excel writer and output the Excel file.
    writer.save()
        
    # Pause the loop
    sleep(random.randint(1,8))
    
    # Monitor the requests
    count += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(count, count/elapsed_time))
#     clear_output(wait = True)
    print('page: ', page)
    
         
    # Update the search page
    seek_url = main_url + search_word + "-jobs"  + '?page='+ str(page+1)
    
    # Get the next page and parse the page, Throw a warning for non-200 status codes
    try:
        seek_page = requests.get(seek_url, headers = header)
        if seek_page.status_code == 200:
            # extract
            seek_soup = bs(seek_page.text,"lxml") 
            print('update page')
        else:
            print(seek_page.status_code)
            # notify, try again
    except requests.Timeout as e:
        print("It is time to timeout")
        print(str(e))

    clear_output(wait = True)
